# 🏥 Breast Cancer Detection - Transfer Learning (DenseNet121)
Ce notebook utilise l'architecture **DenseNet121**, reconnue pour ses performances supérieures en imagerie médicale.

### 🚀 Stratégie :
1.  **Phase 1 (Warmup)** : On gèle le corps du modèle DenseNet121 et on entraîne les couches finales.
2.  **Phase 2 (Fine-tuning)** : On débloque toutes les couches pour affiner la détection des micro-détails mammographiques.

## 🛠️ 1. Installation et Imports

In [ ]:
import os
import shutil
import json
import random
from pathlib import Path
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tqdm.notebook import tqdm

print(f"TensorFlow version: {tf.__version__}")

## ⚙️ 2. Configuration

In [ ]:
CONFIG = {
    'data': {
        'raw_input': 'data/raw',
        'cleaned_output': 'data/cleaned',
        'train_dir': 'data/cleaned/train',
        'val_dir': 'data/cleaned/val',
        'test_dir': 'data/cleaned/test'
    },
    'model': {
        'img_size': 128,
        'base_model': 'DenseNet121',
        'fine_tune_epochs': 15
    },
    'training': {
        'batch_size': 32,
        'epochs': 30,
        'learning_rate': 0.0001
    }
}

## 🧹 3. Preprocessing
Uploadez vos dossiers `Positive` et `Negative` dans `data/raw/` dans la barre latérale gauche.

In [ ]:
def validate_and_clean_image(image_path, target_size=(128, 128)):
    try:
        img = Image.open(image_path)
        img = img.convert("RGB")
        img = img.resize(target_size, Image.Resampling.LANCZOS)
        return img
    except: return None

def prepare_dataset(input_dir, output_dir, target_size=(128, 128), split_ratios=(0.7, 0.15, 0.15)):
    input_path, output_path = Path(input_dir), Path(output_dir)
    for split in ['train', 'val', 'test']:
        for cls in ['Positive', 'Negative']: (output_path / split / cls).mkdir(parents=True, exist_ok=True)
    
    for cls in ['Positive', 'Negative']:
        cls_dir = input_path / cls
        if not cls_dir.exists(): continue
        images = [f for f in cls_dir.glob('*') if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
        random.shuffle(images)
        n = len(images); n1 = int(n * split_ratios[0]); n2 = int(n * (split_ratios[0]+split_ratios[1]))
        
        splits = {'train': images[:n1], 'val': images[n1:n2], 'test': images[n2:]}
        for s_name, s_imgs in splits.items():
            print(f"Processing {s_name}/{cls}...")
            for img_p in s_imgs:
                c_img = validate_and_clean_image(img_p, target_size)
                if c_img: c_img.save(output_path / s_name / cls / f\"{img_p.stem}.jpg\", \"JPEG\")

for d in ['data/raw/Positive', 'data/raw/Negative']: Path(d).mkdir(parents=True, exist_ok=True)
print("🚀 Dossiers initialisés.")

## 🏗️ 4. Modèle (Transfer Learning DenseNet121)

In [ ]:
def build_densenet_model(input_shape=(128, 128, 3)):
    base_model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model, base_model

model, base_model = build_densenet_model()
model.summary()

## 🏋️ 5. Entraînement

In [ ]:
prepare_dataset(CONFIG['data']['raw_input'], CONFIG['data']['cleaned_output'])
img_size = CONFIG['model']['img_size']

def get_ds(path): 
    ds = keras.preprocessing.image_dataset_from_directory(path, image_size=(img_size, img_size), label_mode='binary', batch_size=32)
    return ds.map(lambda x, y: (layers.Rescaling(1./255)(x), y)).cache().prefetch(tf.data.AUTOTUNE)

train_ds = get_ds(CONFIG['data']['train_dir'])
val_ds = get_ds(CONFIG['data']['val_dir'])
test_ds = get_ds(CONFIG['data']['test_dir'])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=CONFIG['training']['learning_rate']),
              loss='binary_crossentropy', metrics=['accuracy'])

print("\n🔥 Phase 1 : Entraînement Initial...")
model.fit(train_ds, validation_data=val_ds, epochs=CONFIG['training']['epochs'])

print("\n❄️  Phase 2 : Fine-tuning...")
base_model.trainable = True
model.compile(optimizer=keras.optimizers.Adam(learning_rate=CONFIG['training']['learning_rate'] / 10),
              loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=CONFIG['model']['fine_tune_epochs'])

model.save('model_breast_cancer_densenet.h5')
print("✅ Modèle final sauvegardé !")